# ABACUS+DPMD+Dflow：Quick Test Semiconductor Melting Point Properties with Candela Installation and Usage Instructions


<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:1rem">
<p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
    ©️ <b><i>Copyright 2023 @ Authors</i></b><br/>
    <i>Author: Mohan Chen Research Group, College of Engineering, Peking University <a href="2000011003@stu.pku.edu.cn"><b>Zhang Xingchen 📨 </b></a></i><br/> 
    <i>Last Updated: 2023-11-14</i><br/>
    <i>AI4Science 3rd Hackathon Challenge Application Scenarios and Exploration Track</i><br/>
    <i>License: This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.</i>
</p>
</div>
<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-top:1rem; margin-bottom:1rem">
<p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
    📖 <b>Getting Started Guide</b><br/>
    This document can be directly run on <a style="font-weight:bold; color:purple" href="https://bohrium-doc.dp.tech/docs/userguide/Notebook" target="_blank">Bohrium Notebook</a>. You can click the button <span style="background-color:darkblue; color:white; padding: 1px; border-radius: 5px;">
    Start Connection</span> at the top of the interface, select the <u>ABACUS:3.3.2-user-guide</u> image and <u>c2_m4_cpu</u> node configuration, and wait a moment to run.
</p>
</div>

# 1. Introduction

### Introduction to ABACUS Molecular Dynamics

ABACUS's molecular dynamics functionality supports both first-principles molecular dynamics (FPMD) method and classical Lennard-Jones (LJ potential) molecular dynamics simulation. In addition, ABACUS also supports Deep Potential Molecular Dynamics (DPMD) method, which requires compiling the DeePMD-kit software package and linking the dynamic library when compiling the atomic calculation software.

After testing, using the image registry.dp.tech/dptech/prod-12058/abacus-deepmd-kit-candela:abacus-deepmd-kit-candela_v1.01, you can directly run the tasks of this tutorial from the command line.

If readers use the dflow script provided below to submit tasks, there is no need to install ABACUS and compile the interface locally, nor to install and compile Candela. Only the ABACUS input file needs to be prepared.

### Introduction to Melting Point Testing Methods
#### Direct Heating Method
- The melting process of a material is observed by heating it directly. At the beginning of the simulation, the atoms or molecules in the system are placed at an initial temperature and the temperature is gradually increased by applying heat. As the temperature increases, the interactions between the atoms or molecules change until the melting point is reached. By monitoring the changes in the position, energy, and other physical properties of the atoms or molecules in the system, the melting point of the material can be determined.

- The improved direct heating method is used in this tutorial. In the traditional direct heating method, where the initial and final temperatures are set differently, each point in the process is in a non-equilibrium state, making it difficult to accurately describe the state of the material at that temperature. Therefore, in this method, the NPT ensemble is used to simulate the system at each temperature for tens of picoseconds to reach an equilibrium state, and the mean square displacement and diffusion coefficient are output to determine the state of the material. However, without using dflow, this method requires manually modifying the parameters and submitting the tasks multiple times, which is cumbersome. The provided dflow script can automate the process to some extent.
#### Two-Phase Method
- The transformation process between the solid and liquid states is observed. At the beginning of the simulation, the atoms or molecules in the system are placed at a low temperature and form a solid structure. Then, by gradually increasing the temperature, the interactions between the atoms or molecules change, causing the material to transition from the solid state to the liquid state. By monitoring the changes in the atomic or molecular structure, density, energy, and other physical properties of the system, the melting point of the material can be determined.
#### Z-Method
- The Z-method is a thermal analysis technique used in molecular dynamics simulations, combining the advantages of the direct heating method and the two-phase method. Continuously changing the initial energy value of the NVE ensemble, after running for a period of time, a balanced T and P value will be obtained, drawn as an isocapacity line and presented as a z-type. The lower vertex of the z-type is the melting point temperature, and the upper vertex is the superheat temperature.

# 2. Preparation

## Environment Configuration
### Online Deployment (for notebook testing only)
Install dflow and Bohrium client:

In [1]:
! pip install pydflow -U -i https://pypi.tuna.tsinghua.edu.cn/simple
! pip install lbg -U -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


Configure the workflow server with the Bohrium account.

In [2]:
import getpass
from dflow import config, s3_config
from dflow.plugins import bohrium
from dflow.plugins.bohrium import TiefblueClient
config["host"] = "https://workflows.deepmodeling.com"
config["k8s_api_server"] = "https://workflows.deepmodeling.com"
bohrium.config["username"] = getpass.getpass("Bohrium username: ")
bohrium.config["password"] = getpass.getpass("Bohrium password: ")
bohrium.config["project_id"] = getpass.getpass("Bohrium project_id: ")
s3_config["repo_key"] = "oss-bohrium"
s3_config["storage_client"] = TiefblueClient()

Bohrium username:  ········
Bohrium password:  ········
Bohrium project_id:  ········


# Example download link

In [3]:
! git clone https://github.com/xinchenniny/ABACUS-dflow.git

fatal: destination path 'ABACUS-dflow' already exists and is not an empty directory.


## Prepare input file INPUT

The example system uses cubic ZnS, and its experimental melting point is 1350 ℃. In this case, we use a simple direct heating method. Previous LAMMPS simulations on a 216-atom system showed that the melting point of ZnS is between 1967~2009K, while the experimental melting point is 1623K, which is about 370K higher.

We can modify the parameter md_tfirst and run the appropriate number of MD steps at each temperature (set by md_nstep, which is an empirical value) to obtain the equilibrium structure at that temperature. When the melting point is unknown, we can select temperature points at equal intervals for testing. After determining the range where the melting point is located, we can insert temperature points at equal intervals within this range to further narrow down the range of melting point testing results. It is important to note that the initial temperature should not exceed the upper limit of the DPGEN sampling temperature.

The following is an example of the file content:

In [4]:
! cat ABACUS-dflow/ABACUS-workflow/DPMD/INPUT

INPUT_PARAMETERS
suffix              DPMD-melting
calculation         md
esolver_type        dp
pot_file            ./frozen_model.pb
ntype               2
md_nstep            25000
md_type             npt
md_pmode            iso
md_pcouple          xyz
md_pfirst           0.001
md_pfreq            0.0005
md_pchain           3
md_tchain           3
md_dt               2
md_tfirst           2500
md_seed             2121491
md_tfreq            0.005
md_dumpfreq         1
md_restartfreq      1000
init_vel            0
dump_force          1
dump_vel            1
dump_virial         1


____

These parameters are all explained in the ABACUS online documentation and are briefly summarized here:

- **calculation**: Set the ABACUS calculation type, set to md for molecular dynamics simulation.
- **esolver_type**: Specify the calculation of system energy given atomic positions, default is Kohn-Sham density functional theory (ksdft), can also be set to LJ potential (lj) or deep potential (dp). In theory, all esolver_type can be used for MD calculation.
- **pot_file**: Path to the potential function file.
- **md_nstep**: Total number of steps for MD simulation.
- **md_type**: Type of MD algorithm, default is NVT for canonical ensemble, this example selects NPT for NVE microcanonical ensemble.
- **md_dt**: Time step for each MD calculation step (in fs), together with md_nstep determines the total duration of MD simulation.
- **md_tfirst**: Initial temperature of the MD system (in K).
- **md_tchain**: Default value is 1 for ABACUS, 3 for LAMMPS.
- **md_tfreq**: Controls the rate of temperature change in MD simulation, recommended value is 1/40/dt. Relationship with Tdamp in LAMMPS: 1/(1000*Tdamp)fs-1.
- **md_seed**: Whether the initial velocities are random.
- **md_pfirst**: Initial pressure of the MD system (in kbar).
- **md_pfreq**: Controls the rate of pressure change in MD simulation, recommended value is 1/400/dt. Relationship with Pdamp in LAMMPS: 1/(1000*Pdamp)fs-1.
- **md_pchain**: Default value is 1 for ABACUS, 3 for LAMMPS.
- **md_pmode**: Method of applying pressure in MD simulation.
- **md_pcouple**: Controls the independence of box scaling direction in MD simulation.
- **md_dumpfreq**: Output frequency of atomic and cell information in the MD output file MD_dump.
- **md_restartfreq**: Output frequency of structure file STRU_MD_${istep}, update frequency of MD restart file Restart_md.dat.

Note: The results of the literature generally melt within 10ps, so it doesn't make much sense to set md_nstep too large.

----

## Preparing the structure file STRU for atomic systems

The files that describe the atomic structure required for this step are:

    *.cif

The structure files can be downloaded from multiple sources. In this case, we mainly download cif files from the materials project website. The semiconductor used may correspond to multiple atomic configurations. Previous LAMMPS simulations have shown that the selection of the initial configuration has little impact on the results, as the equilibrium structure of the system at this temperature exists. The choice of configurations depends on the system and properties being studied. Generally, structures that may appear in the temperature and pressure range of interest can be selected, or the simplest cubic structure can be used. In this case, a cubic ZnS structure is used for demonstration purposes.

Considering the size effect, you can first use VESTA to enlarge the lattice of the cif file to the desired size.

After obtaining the cif file, you can use the following methods to convert the cif file to STRU file:

In [5]:
! git clone https://gitlab.com/1041176461/ase-abacus.git
%cd ase-abacus
! python3 setup.py install

Cloning into 'ase-abacus'...
remote: Enumerating objects: 137072, done.
remote: Counting objects: 100% (14568/14568), done.
remote: Compressing objects: 100% (790/790), done.
remote: Total 137072 (delta 13955), reused 13860 (delta 13778), pack-reused 122504
Receiving objects: 100% (137072/137072), 30.98 MiB | 3.99 MiB/s, done.
Resolving deltas: 100% (107596/107596), done.
Updating files: 100% (1595/1595), done.
/data/ase-abacus
running install
/opt/mamba/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/opt/mamba/lib/python3.10/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating ase.egg-info
writing ase.egg-info/PKG-INFO
writing dependency_links to ase.egg-info/depen

In [6]:
%cd ../ABACUS-dflow/
! ls

/data/ABACUS-dflow
ABACUS-workflow  Candela  STRU	ZnS.cif  candela-test  running0.log


cif to STRU

In [7]:
from ase.io import read, write
from pathlib import Path

cs_dir = './'
cs_vasp = Path(cs_dir, 'ZnS.cif') # Change to your own cif file name
cs_atoms = read(cs_vasp, format='cif')
cs_stru = Path(cs_dir, 'STRU')
write(cs_stru, cs_atoms, format='abacus')

Run the above python script to convert cif to STRU file required by ABACUS.

Note: You can determine whether the keyword "type_map" exists in the DP potential file using the following command.

In [8]:
! strings ./ABACUS-workflow/DPMD/frozen_model.pb | grep type_map

{"model":{"type_map":["Zn","S"],"descriptor":{"type":"se_e2_a","sel":[76,76],"rcut_smth":0.5,"rcut":8.0,"neuron":[25,50,100],"resnet_dt":false,"axis_neuron":12,"seed":24105129190,"activation_function":"tanh","type_one_side":false,"precision":"default","trainable":true,"exclude_types":[],"set_davg_zero":false},"fitting_net":{"neuron":[240,240,240],"resnet_dt":false,"seed":30715280817,"type":"ener","numb_fparam":0,"numb_aparam":0,"activation_function":"tanh","precision":"default","trainable":true,"rcond":0.001,"atom_ener":[]},"data_stat_nbatch":10,"data_stat_protect":0.01},"learning_rate":{"type":"exp","start_lr":0.001,"stop_lr":3.5e-08,"decay_steps":40000,"scale_by_worker":"linear"},"loss":{"start_pref_e":0.2,"limit_pref_e":1,"start_pref_f":1000,"limit_pref_f":1,"start_pref_v":0.01,"limit_pref_v":1.0,"type":"ener","start_pref_ae":0.0,"limit_pref_ae":0.0,"start_pref_pf":0.0,"limit_pref_pf":0.0,"enable_atom_ener_coeff":false},"training":{"disp_file":"lcurve.out","disp_freq":100,"save_freq

If the keyword type_map exists, ABACUS will automatically match the atomic species order in the STRU and DP potential files.

Otherwise, the atomic species and order in STRU must be consistent with those in the DP potential files.

## Prepare DP potential function file frozen_model.pb

The selected DP potential function in this case is derived from ABACUS+DPGEN. If readers are interested in the generation method of the potential function, they can refer to the ABACUS Chinese documentation tutorial.

## Preparation for Candela

Note: This section is not relevant to the task execution and is only for readers who need to install Candela. You can skip this section if you are following the tasks in order.

### Introduction
Candela, short for Collection of ANalysis DEsigned for Large-scale Atomic simulations, currently supports the analysis of molecular dynamics trajectories from QE, ABACUS, LAMMPS, and VASP. The Github homepage can be found at: https://github.com/MCresearch/Candela

For more information on using ABACUS+Candela, please refer to: https://mcresearch.gitee.io/abacus-user-guide/abacus-candela.html

We use Candela for post-processing tasks.

### Environment Preparation

The image "registry.dp.tech/dptech/prod-12058/abacus-deepmd-kit-candela:abacus-deepmd-kit-candela_v1.00" provided in the dflow script has already installed Candela. You can directly run the command "candela" in the command line.

If Candela is not installed in the runtime environment, manual installation is required. Follow the steps below:

In [9]:
! git clone https://github.com/MCresearch/Candela.git
! cd Candela && make CXX=g++ TEST=ON

fatal: destination path 'Candela' already exists and is not an empty directory.
make: 'bin/candela' is up to date.


At this time, Candela compilation is completed and the serial version can be run, which is sufficient for calculating MSD for general systems.

After preparing the input file and MD_dump file, go to the path where the input file is located and run the command line directly to obtain the output result.

Note:

If using the Intel Oneapi compiler, the compilation can be done using the command "make -j4". After the compilation is completed, the executable file "candela" can be found in the bin directory.

If using other compilers, you need to modify the corresponding CXX in the Makefile.vars under the Candela directory.

### Input File Preparation

The example INPUT file is as follows:

(Note that the unit of msd_dt here is ps, which needs to be modified to match the md_dt in the ABACUS INPUT file)
```
calculation  msd # Pair Distribution Function.
system ZnS
geo_in_type  ABACUS
geo_directory MD_dump
geo_1        1
geo_2        2000
geo_interval 1
geo_ignore   300

ntype        2        # number of different types of atoms.
natom        216   # total number of atoms.
natom1       108   #Zn
natom2       108   #S
id1          Zn
id2          S
msd_dt       0.002
```

# 3. Building and Running Dflow

## Flowchart
Prepare input file - [Modify md_tfisrt - Submit task - Post-processing with candela to obtain MSD]

For testing the melting point properties of semiconductors, it is most flexible to execute the parts inside the box in parallel using dflow. Here, a Slices OP is used to implement this.

The improved flowchart is shown below:

![image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/12058/38970301ae5149a3b11fe5bc684c1397/Ldimw09TcX1HW5l9IJMWhQ.png)


## Dflow Code
Note: The number of temperature points NUM needs to be manually modified. The input file is a packaged ABACUS input folder (named DPMD here) and md_tfirst_file. This script should be executed in the same directory. This code only needs to be run once. If the example python code has been run, there is no need to repeat the subsequent steps.

In [10]:
%cd ../ABACUS-dflow/ABACUS-workflow

/data/ABACUS-dflow/ABACUS-workflow


In [11]:
NUM = 3 #The number of temperature values to calculate
from dflow.plugins.dispatcher import DispatcherExecutor

dispatcher_executor = DispatcherExecutor(
        machine_dict={
            "batch_type": "Bohrium",
            "context_type": "Bohrium",
            "remote_profile": {
                "input_data": {
                    "job_type": "container",
                    "platform": "ali",
                    "scass_type" : "c8_m16_cpu"
                },
            },
        },
    )

from typing import List
from dflow import Step, Workflow
from dflow.python import OP, OPIO, Artifact, OPIOSign, PythonOPTemplate, Slices
from dflow.utils import upload_artifact, download_artifact
import os
from pathlib import Path
import subprocess
import shutil
import numpy as np


class DPMD(OP):
    def __init__(self):
        pass
         
    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            "ABACUS_INPUT": Artifact(Path),  # a series of input files
            "md_tfirst_file":Artifact(Path), # the path of the temperature file
            "index": int
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            "output": Artifact(Path)  # the path of the OUT.suffix file
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        cwd = os.getcwd()
        '''
        # Get the directory path of the "md_tfirst_file"
        md_tfirst_file_dir = os.path.dirname(op_in["md_tfirst_file"])

        # Change the current working directory to the directory containing the "md_tfirst_file"
        os.chdir(md_tfirst_file_dir)
        '''
        md_tfirst_file_path = os.path.join(op_in["md_tfirst_file"])
        index = op_in["index"]
        # Read the new value from the "md_tfirst_file"
        with open(md_tfirst_file_path, "r") as f:
            line = f.readline().strip()  # Read the first line and strip whitespace characters
            values = line.split()  # Split the line into a list of values
            new_md_tfirst = values[index]  # Get the second value (index 1)
        
        os.chdir(op_in["ABACUS_INPUT"] )
        # Change the current working directory to the ABACUS_INPUT folder
        input_file_path = os.path.join(op_in["ABACUS_INPUT"], "INPUT")
    
        # Concatenate the ABACUS_INPUT folder path and "INPUT" to create the input_file_path
        with open(input_file_path, "r") as f:
            content = f.read()

       # Find the line where the md_tfirst parameter is located
        lines = content.splitlines()
        for i, line in enumerate(lines):
            if "md_tfirst" in line:
                # Split the line into parameter name and parameter value
                parts = line.split()
                param_name = parts[0]
                old_value = parts[1]

                # Replace the parameter value with the new value
                new_value = new_md_tfirst  # Set the new value for md_tfirst
                parts[1] = new_value

                # Reassemble the modified line into a string
                new_line = " ".join(parts)
                
                # Replace the original line with the new line
                lines[i] = new_line
                break
    
        # Update file content
        new_content = "\n".join(lines)
    
        # Save modified file content
        with open(input_file_path, "w") as f:
             f.write(new_content)
    
        # Execute computing task
        cmd =  "abacus" 
        subprocess.call(cmd, shell=True)
        os.chdir("./OUT.DPMD-melting")
        
        # Create a new folder named myCandela
        new_folder = "myCandela"
        if not os.path.exists(new_folder):
            os.mkdir(new_folder)

        # Copy the MD_dump file to the myCandela folder
        old_file = "MD_dump"
        new_path = os.path.join(new_folder, old_file)
        shutil.copy2(old_file, new_path)
        os.chdir(new_folder)
        # Execute the calculation task
        cmd =  "tree" 
        subprocess.call(cmd, shell=True)

        # Prepare the Candela input file
        content = '''
        calculation  msd # Pair Distribution Function.
        system ZnS
        geo_in_type  ABACUS
        geo_directory MD_dump
        geo_1        1
        geo_2        25000
        geo_interval 1
        geo_ignore   0

        ntype        2        # number of different types of atoms.
        natom        216   # total number of atoms.
        natom1       108   #Zn
        natom2       108   #S
        id1          Zn
        id2          S
        msd_dt       0.002
        '''
        with open("INPUT", "w") as file:
            file.write(content)

        # Execute the calculation task
        cmd =  "candela" 
        subprocess.call(cmd, shell=True)

        # Read the MSD.dat file
        data = np.loadtxt('MSD.dat')

        # Extract time and mean square displacement data
        time = data[:, 0]
        msd = data[:, 1]

        # Calculate the diffusion coefficient
        slope, intercept = np.polyfit(time, msd, 1)
        diffusion_coefficient = slope / 6

        # Write the diffusion coefficient to a file
        filename = str(new_md_tfirst)+"K_result.txt"
        file_path = Path(filename)
        with file_path.open("w") as file:
            file.write(str(diffusion_coefficient))
        print("Temperature：", new_md_tfirst,"K")
        print("Diffusion coefficient：", diffusion_coefficient)
        return OPIO({
            "output":file_path
        })       



from dflow import Workflow
wf = Workflow(name = "my-abacus-dpmd-workflow")
DPMD = Step(name = "DPMD",
            template = PythonOPTemplate(DPMD,image ="registry.dp.tech/dptech/prod-12058/abacus-deepmd-kit-candela:abacus-deepmd-kit-candela_v1.01",
            slices=Slices("{{item}}",
                    input_parameter=["index"],
                    output_artifact=["output"]
                )),
            parameters={"index": [x for x in range(NUM)]},
            with_param=range(NUM),
            artifacts={ "ABACUS_INPUT": upload_artifact("DPMD"),
                        "md_tfirst_file":upload_artifact("md_tfirst_file")},
            executor = dispatcher_executor)
wf.add(DPMD)
wf.submit()

Workflow has been submitted (ID: my-abacus-dpmd-workflow-6j8v2, UID: 05a9df00-9344-4d04-96eb-53f999a5ffe4)
Workflow link: https://workflows.deepmodeling.com/workflows/argo/my-abacus-dpmd-workflow-6j8v2


{'metadata': {'name': 'my-abacus-dpmd-workflow-6j8v2', 'generateName': 'my-abacus-dpmd-workflow-', 'namespace': 'argo', 'uid': '05a9df00-9344-4d04-96eb-53f999a5ffe4', 'resourceVersion': '239779056', 'generation': 1, 'creationTimestamp': '2023-11-14T07:56:16Z', 'labels': {'bohrium.dp.tech/project-id': '15069', 'bohrium.dp.tech/user-id': '12058', 'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo'}, 'annotations': {'bohrium.dp.tech/email': '2000011003@stu.pku.edu.cn', 'bohrium.dp.tech/project-id': '15069', 'bohrium.dp.tech/user-id': '12058'}, 'managedFields': [{'manager': 'argo', 'operation': 'Update', 'apiVersion': 'argoproj.io/v1alpha1', 'time': '2023-11-14T07:56:16Z', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:annotations': {'.': {}, 'f:bohrium.dp.tech/email': {}, 'f:bohrium.dp.tech/project-id': {}, 'f:bohrium.dp.tech/user-id': {}}, 'f:generateName': {}, 'f:labels': {'.': {}, 'f:bohrium.dp.tech/project-id': {}, 'f:bohrium.dp.tech/user-id': {}, 'f:workflows

### Function Introduction:
Read the input file required by ABACUS, write the temperatures to be tested in **md_tfrist_file**, modify the value of **NUM** in the script, and directly obtain the diffusion coefficient at each temperature.

Note that you need to check whether the INPUT file and the INPUT file setting of Candela in the script are correct. Currently, if the INPUT parameters of Candela are set incorrectly, there will be no error and it will automatically jump out. It will be stuck at the next step of reading the MD_dump file. If you encounter a problem where the task does not end for a long time, it is recommended to monitor the log file on Bohrium.

## Running and Result Analysis

### Running
Since Python code has been run before, there is no need to run it again.

In [12]:
# python abacusworkflow.py

### Result Analysis

You can monitor the completion of the task on Bohrium and see that the final output is 

![图片.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/12058/467917e4d29e45b5b6d6657bbcc6a4cf/QMgH7ey1TdhvAuiDgYb3Ug.png)

to confirm that the substance is in solid state at the current temperature.

In fact, it is not enough to judge only by the diffusion coefficient. The abnormality of MSD can also lead to abnormal diffusion coefficients. It is recommended to check the MSD.dat file in the output and plot it.

- This is a normal MSD

![Image1.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/12058/43a1436290d6473e86a9f1a112162bd8/eDwLjzzbmwkfnFhkJXAzOQ/KYJQhLPGGkBRgjHOA9cnjg.png)

- This is an abnormal MSD

![Image2.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/12058/43a1436290d6473e86a9f1a112162bd8/2GqRV24kUb_D7CN8u1KDHA/1cl6bZXFRr1ThjIjihNf0w.png)

Obviously, it is not enough to judge the normality of the melting point position solely based on the diffusion coefficient. In case of abnormal situations, careful investigation is required.

## Feasible Improvement Solutions

- If the investigation of MSD and abnormal diffusion coefficient is not considered, this process can be recursive until the upper and lower bounds of the melting point output meet the accuracy requirements.
- If the future potential function can be directly downloaded from the website, the process of preparing input files can be automated without manual preparation.
- Not only the melting point test can be efficiently implemented using workflows, but also other common test properties can be met with the engineering requirements of testing using workflows, such as the method and workflow for thermal conductivity calculation, which is currently under development and can be written as a Notebook in the future.

## Reference
### ABACUS Online Documentation
ABACUS Chinese documentation address: [https://mcresearch.gitee.io/abacus-user-guide/](https://mcresearch.gitee.io/abacus-user-guide/)
### Literature on Melting Point Calculation:
- Direct Heating Method
- Raty, J. Y., Schwegler, E., & Bonev, S. A. (2008). Electronic and structural transitions in dense liquid sodium. Nature, 451(7175), 676-678.
- Tamblyn, I., Raty, J. Y., & Bonev, S. A. (2008). Tetrahedral Clustering in Molten Lithium under Pressure. Physical Review Letters, 101(7), 075703. doi: 10.1103/PhysRevLett.101.075703.
- Two-Phase Method
- Morris, J. R., & Song, X. (2002). The melting lines of model systems calculated from coexistence simulations. Journal of Chemical Physics, 116(21), 9352. doi: 10.1063/1.1474581.
- Alfe`, D. (2003). First-principles simulations of direct coexistence of solid and liquid aluminum. Physical Review B, 68(6), 064423. doi: 10.1103/PhysRevB.68.064423.
- Herna´ndez, E. R., Rodriguez-Prieto, A., Bergara, A., & Alfe`, D. (2010). First-Principles Simulations of Lithium Melting: Stability of the bcc Phase Close to Melting. Physical Review Letters, 104(17), 175702. doi: 10.1103/PhysRevLett.104.175702.
- Z-Method
- Belonoshko, A. B., & Rosengren, A. (2012). High-pressure melting curve of platinum from ab initio Z method. Physical Review B, 85(17), 174104. doi: 10.1103/PhysRevB.85.174104.
- Thermodynamic Integration Method for Melting Point Calculation
- Jesson, B. J., & Maddena, P. A. (2000). Ab initio determination of the melting point of aluminum by thermodynamic integration. Journal of Chemical Physics, 113(5), 1963-1966.
- Vocˇadlo, L., & Alfe`, D. (2002). Ab initio melting curve of the fcc phase.